In [42]:
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Flatten,Softmax,Embedding,Reshape,SimpleRNN,GRU,LSTM
import numpy as np

tf.__version__, K.__version__

('1.9.0', '2.1.6-tf')

In [19]:
tf.reset_default_graph()
sentences = ['i like dog', 'i love coffee', 'i hate milk']

word_list = ' '.join(sentences).split()
word_list = list(set(word_list))
word_dict = {w:i for i,w in enumerate(word_list)}
number_dict = {i:w for i,w in enumerate(word_list)}
n_class = len(word_dict)

def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split()
        input_ = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]
        
        input_batch.append(np.eye(n_class)[input_])
        # one-hot label
        target_batch.append(np.eye(n_class)[target])
        # target_batch.append(target)
    
    return np.array(input_batch), np.array(target_batch)

input_batch, target_batch = make_batch(sentences)

Train the RNN model:  


In [47]:
# TextRNN Parameter
n_step = 2 # number of cells (=number of Step)
n_hidden = 5 # number of hidden units in on cell

model = Sequential()
model.add(LSTM(n_hidden))# output shape [batch_size, n_step, n_hidden])
model.add(Dense(n_class,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [48]:
model.fit(input_batch, target_batch, epochs=100)

Epoch 1/100
3/3 [==============================] - 1s 297ms/step - loss: 1.9452 - acc: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 0s 1ms/step - loss: 1.9421 - acc: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 1.9391 - acc: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 0s 1ms/step - loss: 1.9360 - acc: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 0s 1ms/step - loss: 1.9330 - acc: 0.0000e+00
Epoch 6/100
3/3 [==============================] - 0s 1ms/step - loss: 1.9299 - acc: 0.0000e+00
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 1.9269 - acc: 0.3333
Epoch 8/100
3/3 [==============================] - 0s 1ms/step - loss: 1.9238 - acc: 0.3333
Epoch 9/100
3/3 [==============================] - 0s 1ms/step - loss: 1.9208 - acc: 0.3333
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: 1.9177 - acc: 0.3333
Epoch 11/100
3/3 [==============================] - 0

Epoch 89/100
3/3 [==============================] - 0s 891us/step - loss: 1.6726 - acc: 0.3333
Epoch 90/100
3/3 [==============================] - 0s 2ms/step - loss: 1.6694 - acc: 0.3333
Epoch 91/100
3/3 [==============================] - 0s 1ms/step - loss: 1.6662 - acc: 0.3333
Epoch 92/100
3/3 [==============================] - 0s 1ms/step - loss: 1.6629 - acc: 0.3333
Epoch 93/100
3/3 [==============================] - 0s 2ms/step - loss: 1.6597 - acc: 0.3333
Epoch 94/100
3/3 [==============================] - 0s 1ms/step - loss: 1.6564 - acc: 0.3333
Epoch 95/100
3/3 [==============================] - 0s 969us/step - loss: 1.6532 - acc: 0.3333
Epoch 96/100
3/3 [==============================] - 0s 3ms/step - loss: 1.6499 - acc: 0.3333
Epoch 97/100
3/3 [==============================] - 0s 1ms/step - loss: 1.6467 - acc: 0.3333
Epoch 98/100
3/3 [==============================] - 0s 1ms/step - loss: 1.6434 - acc: 0.3333
Epoch 99/100
3/3 [==============================] - 0s 1ms/step - 

In [49]:
input_ = np.array([sen.split()[:2] for sen in sentences])

predict = model.predict(input_batch)
predict = np.argmax(predict, axis=1)
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n] for n in predict])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'dog', 'dog']


- 输入维度(3, 2, 7) `# [batch_size, sequence_lens, embedding_size]`没有使用嵌入层，所以`embedding_size` = `vocab_size`  
- RNN`h_t`向量维度为`n_hidden = 5`，所以输出维度为(3, 5) `# [batch_size, n_hidden]`  
    (根据`h_t = tanh(W_x X_t + W_h h_{t-1} + b)`得参数为(7+5)*5+5= 65)  
- 将RNN输出Reshape成维度(3, -1) `# [batch_size, -1]`  
- Dence层输出维度为(3, 7) `# [batch_size, vocab_size]`  

**RNN/GRU/LSTM 参数量计算**:  
SimpleRNN:$m\times n + n\times n + n = (m+n)\times n + n = (m+n+1)*n$  
GRU: 多了两个门，所以是$3\teims$

In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5)                 260       
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 42        
Total params: 302
Trainable params: 302
Non-trainable params: 0
_________________________________________________________________
